Gensim

On Spam CSV

In [ ]:
import numpy as np
import pandas as pd
import re
import string

In [ ]:
!pip install -q wordcloud
import wordcloud 

import nltk  
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

True

In [ ]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from gensim.models import Word2Vec

In [ ]:
import gensim
import multiprocessing

In [27]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import sys
def replace_none(X):
    if X == '':
        X = np.nan
    return X

In [28]:
def build_model(max_epochs, vec_size, alpha, tagged_data):
    
    model = Doc2Vec(vector_size=vec_size,
               alpha=alpha,
               min_alpha=0.00025,
               min_count=1,
               dm=1)
    
    model.build_vocab(tag_data)
      
    for epoch in range(max_epochs):
        print(f"Iteration {epoch}")
        model.train(tag_data,
                   total_examples=model.corpus_count,
                   epochs=model.epochs)

        model.alpha -= 0.0002

        model.min_alpha = model.alpha
    
    model.save("SPAM_w2v_MODEL.model")
    print("Model Saved")
    return model

In [29]:
messages = pd.read_csv('spam.csv', encoding='latin-1')
messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
messages.columns = ["label", "text"]
messages.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [30]:
messages.isnull().sum()

label    0
text     0
dtype: int64

In [36]:
messages['text'] = messages['text'].str.replace('[^a-zA-Z]',' ').str.lower()
stop_re = '\\b'+'\\b|\\b'.join(nltk.corpus.stopwords.words('english'))+'\\b'
messages['text'] = messages['text'].str.replace(stop_re, '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [37]:
w2v_total_data = list(messages['text'])
w2v_total_data

['go  jurong point  crazy   available   bugis n great world la e buffet    cine  got amore wat   ',
 'ok lar    joking wif u oni   ',
 'free entry     wkly comp  win fa cup final tkts   st may       text fa         receive entry question std txt rate  c  apply               ',
 'u dun say  early hor    u c already  say   ',
 'nah    think  goes  usf   lives around  though',
 'freemsg hey  darling      week     word back    like  fun     still  tb ok  xxx std chgs  send          rcv',
 'even  brother   like  speak     treat  like aids patent ',
 ' per  request  melle melle  oru minnaminunginte nurungu vettam     set   callertune   callers  press     copy  friends callertune',
 'winner     valued network customer    selected  receivea       prize reward   claim call              claim code kl     valid    hours  ',
 '  mobile    months    u r entitled  update   latest colour mobiles  camera  free  call  mobile update co free             ',
 '  gonna  home soon     want  talk   stuff anym

In [38]:
tag_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(w2v_total_data)]

In [39]:
model = build_model(max_epochs=5, vec_size=10, alpha=0.025, tagged_data=tag_data)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Model Saved


In [40]:
model.wv.similar_by_word("message")

[('anonymous', 0.9147464036941528),
 ('hol', 0.8983598947525024),
 ('forwarded', 0.8791924715042114),
 ('inviting', 0.8668359518051147),
 ('toll', 0.8604133129119873),
 ('msgs', 0.86033034324646),
 ('yes', 0.8561614751815796),
 ('forgive', 0.851171612739563),
 ('collected', 0.8483546376228333),
 ('fones', 0.847987949848175)]

In [41]:
model.wv.similar_by_word("text")

[('xafter', 0.9165717363357544),
 ('link', 0.8988628387451172),
 ('inviting', 0.8951982259750366),
 ('wild', 0.8908042907714844),
 ('extra', 0.8872353434562683),
 ('daily', 0.8864933252334595),
 ('charge', 0.8859941959381104),
 ('picsfree', 0.8853920102119446),
 ('mix', 0.8823180794715881),
 ('chatting', 0.8728373646736145)]

In [42]:
messages = pd.read_csv('spam.csv', encoding='latin-1')
messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
messages.columns = ["label", "text"]
messages.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [43]:
messages['text'] = messages['text'].str.split()
messages['text'].head()

0    [Go, until, jurong, point,, crazy.., Available...
1                 [Ok, lar..., Joking, wif, u, oni...]
2    [Free, entry, in, 2, a, wkly, comp, to, win, F...
3    [U, dun, say, so, early, hor..., U, c, already...
4    [Nah, I, don't, think, he, goes, to, usf,, he,...
Name: text, dtype: object

In [44]:
phrases = gensim.models.phrases.Phrases(messages['text'].tolist())
phraser = gensim.models.phrases.Phraser(phrases)
messages_phrased = phraser[messages['text'].tolist()]

In [45]:
multiprocessing.cpu_count()

2

In [46]:
w2v = gensim.models.word2vec.Word2Vec(sentences=messages_phrased,workers=2)
w2v.most_similar('message',topn=10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('or', 0.9998729825019836),
 ('our', 0.9998661279678345),
 ('was', 0.9998623132705688),
 ('get', 0.9998606443405151),
 ("I'm", 0.9998601078987122),
 ('with', 0.9998595714569092),
 ('out', 0.9998592138290405),
 ('your', 0.9998579621315002),
 ('Call', 0.9998562335968018),
 ('-', 0.9998550415039062)]

In [47]:
w2v.most_similar('text',topn=10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('send', 0.9999374747276306),
 ('with', 0.9999371767044067),
 ('or', 0.9999361038208008),
 ('be', 0.9999332427978516),
 ('by', 0.999932050704956),
 ('and', 0.9999287128448486),
 ('our', 0.9999282360076904),
 ('more', 0.9999274015426636),
 ('for', 0.9999265670776367),
 ('like', 0.9999258518218994)]

In [48]:
from sklearn.feature_extraction.text import CountVectorizer
stopwords = nltk.corpus.stopwords.words('english')

messages = pd.read_csv('spam.csv', encoding='latin-1')
messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
messages.columns = ["label", "text"]

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word not in stopwords]
    return text

In [49]:
h_vect = CountVectorizer(analyzer=clean_text)
X_h = h_vect.fit_transform(messages['text'])

X_features = pd.DataFrame(X_h.toarray())
X_features.head()

,0,1,2,3,4,5,6,7,8,9,...,9385,9386,9387,9388,9389,9390,9391,9392,9393,9394
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
from sklearn.ensemble import RandomForestClassifier
print(RandomForestClassifier())

RandomForestClassifier()


In [51]:
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_features,messages['label'],test_size=0.2)
rf = RandomForestClassifier()
rf_model = rf.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
precision = precision_score(y_test, y_pred, pos_label='spam')
recall = recall_score(y_test, y_pred, pos_label='spam')
print('Precision: {} / Recall: {}'.format(round(precision, 2), round(recall, 2)))

Precision: 1.0 / Recall: 0.75


***EXTRA:

word2vec On different corpuses in nltk.

In [52]:
from nltk.corpus import webtext, reuters, inaugural 
w = Word2Vec(webtext.sents())
r = Word2Vec(reuters.sents())
i = Word2Vec(inaugural.sents())

In [53]:
w.most_similar('web', topn=15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('multiple', 0.9919770956039429),
 ('list', 0.9919420480728149),
 ('form', 0.988717257976532),
 ('extension', 0.9873952865600586),
 ('url', 0.9854311943054199),
 ('location', 0.9853098392486572),
 ('popup', 0.9844474792480469),
 ('items', 0.9844255447387695),
 ('buttons', 0.984387993812561),
 ('appears', 0.9838602542877197),
 ('sidebar', 0.9838454723358154),
 ('loading', 0.9836207628250122),
 ('source', 0.9835255146026611),
 ('links', 0.9830262660980225),
 ('site', 0.9830076098442078)]

In [54]:
r.most_similar('news', topn=15) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('conference', 0.9273158311843872),
 ('press', 0.9126105308532715),
 ('reporters', 0.8381246328353882),
 ('remarks', 0.8198529481887817),
 ('briefing', 0.8152675032615662),
 ('hearing', 0.7986353039741516),
 ('speaking', 0.7917201519012451),
 ('speech', 0.7882636189460754),
 ('Miyazawa', 0.7766221761703491),
 ('Lawson', 0.7765090465545654),
 ('Stoltenberg', 0.7737005949020386),
 ('House', 0.7716069221496582),
 ('Budget', 0.7686792612075806),
 ('Commons', 0.7636077404022217),
 ('Speaking', 0.755973219871521)]

In [55]:
i.most_similar('beginning', topn=15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('individual', 0.9997798204421997),
 ('where', 0.9997662305831909),
 ('questions', 0.9997621774673462),
 ('very', 0.9997576475143433),
 ('beyond', 0.999752402305603),
 ('true', 0.9997516870498657),
 ('like', 0.9997508525848389),
 ('old', 0.9997495412826538),
 ('whose', 0.9997426271438599),
 ('economy', 0.9997410178184509),
 ('responsibility', 0.9997315406799316),
 ('experience', 0.9997297525405884),
 ('conditions', 0.9997252225875854),
 ('last', 0.9997251033782959),
 ('necessary', 0.9997231960296631)]

Refrences:

https://www.kaggle.com/code/twistedtensor/basic-word2vec-using-gensim/notebook
https://www.kaggle.com/code/fmitchell259/creating-a-doc2vec-model/notebook